In [14]:
import pandas as pd
import numpy as np
import heapq
import math
import time
import os
import gmpy2
from gmpy2 import mpz
import re
from sklearn import tree
import cProfile
from random import randint, sample, seed
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold # import KFold
from osdt import bbound, predict
from memory_profiler import memory_usage

In [15]:
# Below is for rev2: each dataset run 20 times, with 70/30 split for train/test
# init_cart = true meaning that if OSDT fails the CART tree will be returned
source_folder = "data/pydl8.5_datasets"
dest_folder = "data/pydl8.5_datasets_split_20"
log_filename = "dl85_osdt_result_20.txt"

In [5]:
data_files = os.listdir(source_folder)
with open(log_filename, 'a') as logfile:
    logfile.write("name, batch, osdt_accu, osdt_accu_train, osdt_time_elapsed, best_is_cart mem_MiB \n")
for this_file in data_files:  
    filename_stem = this_file.split('.')[0]
    for k in range(1,21):
        X_train = np.genfromtxt(dest_folder + "/" + filename_stem + "_X_train_" + str(k) + ".txt", delimiter = " ").astype(int)
        y_train = np.genfromtxt(dest_folder + "/" + filename_stem + "_y_train_" + str(k) + ".txt", delimiter = " ").astype(int)
        X_test = np.genfromtxt(dest_folder + "/" + filename_stem + "_X_test_" + str(k) + ".txt", delimiter = " ").astype(int)
        y_test = np.genfromtxt(dest_folder + "/" + filename_stem + "_y_test_" + str(k) + ".txt", delimiter = " ").astype(int)
        # Train model using OSDT
        start_time = time.time()
        mem_MiB, (leaves_c, prediction_c, dic, nleaves_OSDT, nrule, ndata, totaltime, time_c, COUNT, C_c, trainaccu_OSDT, best_is_cart, clf) =\
            memory_usage((bbound, (X_train, y_train, 0.005, 'curiosity', float('Inf'), float('Inf'), float('Inf'), False, \
                                   True, True, True, True, True, True, 1, 300, True, False, False)), max_usage=True, retval=True)
        end_time = time.time()
        time_elapsed = end_time - start_time
        # Prediction (copied from the osdt.py)
        _, testaccu_OSDT = predict(leaves_c, prediction_c, dic, X_test, y_test, best_is_cart, clf)
        _, trainaccu_OSDT = predict(leaves_c, prediction_c, dic, X_train, y_train, best_is_cart, clf)
        with open(log_filename, 'a') as logfile:
            logfile.write(filename_stem + ', ' + "{:d}, {:.5f}, {:.5f}, {:.2f}, {:d}, {:.6f}".format(k, testaccu_OSDT, trainaccu_OSDT, time_elapsed, int(best_is_cart), mem_MiB) + '\n')


Current case: letter1
Memory (MB) usage before training:
2491.65625
nrule: 224
ndata: 14000
gr: [9.46585975e-05 9.46585975e-05 1.56232217e-04 1.56232217e-04
 1.34704375e-04 1.34704375e-04 1.11594356e-04 1.11594356e-04
 8.30950586e-05 8.30950586e-05 5.80830785e-05 5.80830785e-05
 7.85023209e-05 7.85023209e-05 1.26800550e-06 1.26800550e-06
 8.89906325e-07 8.89906325e-07 9.73533825e-07 9.73533825e-07
 7.56284252e-07 7.56284252e-07 2.06790330e-08 2.06790330e-08
 1.73247353e-06 1.73247353e-06 3.90738599e-06 3.90738599e-06
 4.32167333e-05 4.32167333e-05 3.79323536e-05 3.79323536e-05
 4.84542609e-05 4.84542609e-05 1.32085500e-05 1.32085500e-05
 1.89332345e-06 1.89332345e-06 3.13817889e-05 3.13817889e-05
 1.49716981e-05 1.49716981e-05 5.85095835e-06 5.85095835e-06
 2.46087931e-06 2.46087931e-06 3.62891610e-06 3.62891610e-06
 7.44304905e-06 7.44304905e-06 1.21811594e-05 1.21811594e-05
 6.78487108e-06 6.78487108e-06 5.65682293e-05 5.65682293e-05
 1.74275501e-06 1.74275501e-06 3.73510496e-05 3.73

IndexError: Replacement index 5 out of range for positional args tuple

In [ ]:
# bbound function declaration in osdt.py
'''
def bbound(x, y, lamb, prior_metric=None, MAXDEPTH=float('Inf'), MAX_NLEAVES=float('Inf'), niter=float('Inf'), logon=False,
           support=True, incre_support=True, accu_support=True, equiv_points=True,
           lookahead=True, lenbound=True, R_c0 = 1, timelimit=float('Inf'), init_cart = True,
           saveTree = False, readTree = False):
'''

In [16]:
# loop content test
filename_stem = "zoo-1"
k = 1
X_train = np.genfromtxt(dest_folder + "/" + filename_stem + "_X_train_" + str(k) + ".txt", delimiter = " ").astype(int)
y_train = np.genfromtxt(dest_folder + "/" + filename_stem + "_y_train_" + str(k) + ".txt", delimiter = " ").astype(int)
X_test = np.genfromtxt(dest_folder + "/" + filename_stem + "_X_test_" + str(k) + ".txt", delimiter = " ").astype(int)
y_test = np.genfromtxt(dest_folder + "/" + filename_stem + "_y_test_" + str(k) + ".txt", delimiter = " ").astype(int)
# Train model using OSDT
start_time = time.time()
mem_MiB, (leaves_c, prediction_c, dic, nleaves_OSDT, nrule, ndata, totaltime, time_c, COUNT, C_c, trainaccu_OSDT, best_is_cart, clf) =\
    memory_usage((bbound, (X_train, y_train, 0.005, 'curiosity', float('Inf'), float('Inf'), float('Inf'), False, \
                           True, True, True, True, True, True, 1, 300, True, False, False)), max_usage=True, retval=True)
end_time = time.time()
time_elapsed = end_time - start_time
print("Run time: {:.1f}, Memory usage (MiB): {:.6f}".format(time_elapsed, mem_MiB))
# Prediction (copied from the osdt.py)
_, testaccu_OSDT = predict(leaves_c, prediction_c, dic, X_test, y_test, best_is_cart, clf)
_, trainaccu_OSDT = predict(leaves_c, prediction_c, dic, X_train, y_train, best_is_cart, clf)

nrule: 36
ndata: 70
gr: [3.31554498e-01 3.31554498e-01 7.10204082e-02 7.10204082e-02
 4.04115646e-01 4.04115646e-01 4.85306122e-01 4.85306122e-01
 8.58163265e-02 8.58163265e-02 5.82331954e-02 5.82331954e-02
 3.06122449e-04 3.06122449e-04 1.66924504e-01 1.66924504e-01
 7.10204082e-02 7.10204082e-02 9.36178108e-02 9.36178108e-02
 3.81405896e-02 3.81405896e-02 2.00000000e-02 2.00000000e-02
 5.64492967e-02 1.59939531e-02 2.40443319e-01 4.97485951e-03
 4.42922976e-02 4.97485951e-03 3.68367347e-02 3.68367347e-02
 1.14581962e-02 1.14581962e-02 1.70665924e-01 1.70665924e-01]
order: [6, 7, 5, 4, 1, 0, 26, 34, 35, 15, 14, 19, 18, 8, 9, 16, 2, 3, 17, 11, 10, 24, 28, 20, 21, 31, 30, 22, 23, 25, 32, 33, 27, 29, 12, 13]
odr: [6, 7, 5, 4, 1, 0, 26, 34, 35, 15, 14, 19, 18, 8, 9, 16, 2, 3, 17, 11, 10, 24, 28, 20, 21, 31, 30, 22, 23, 25, 32, 33, 27, 29, 12, 13]
the order of x's columns:  [6, 7, 5, 4, 1, 0, 26, 34, 35, 15, 14, 19, 18, 8, 9, 16, 2, 3, 17, 11, 10, 24, 28, 20, 21, 31, 30, 22, 23, 25, 32, 33